In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama

agent_model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0.0)

summary_model = ChatOllama(
    model="llama3.2",
)

agent = create_agent(
    model=agent_model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=summary_model,
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [5]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\n## SESSION INTENT\nThe user's primary goal is to gather information about the capital of the moon and its current weather conditions.\n\n## SUMMARY\n* The conversation discussed the capital of the moon (Lunapolis) and its weather conditions (clear skies, 120C high, -100C low).\n* The user asked multiple follow-up questions, including the population size of cheese miners in Lunapolis.\n* The AI provided responses to each question, but it was later revealed that the information about cheese miners is not relevant to the initial query.\n\n## ARTIFACTS\nNone\n\n## NEXT STEPS\nThe specific tasks remaining to be completed include providing accurate and relevant information about the capital of the moon.", additional_kwargs={'lc_source': 'summarization'}, response_metadata={}, id='1d4c26a5-447f-4f0b-883e-be473bbfd38f'),
              HumanMessage(content="If you were Lunapolis' new president how would you re

In [6]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

## SESSION INTENT
The user's primary goal is to gather information about the capital of the moon and its current weather conditions.

## SUMMARY
* The conversation discussed the capital of the moon (Lunapolis) and its weather conditions (clear skies, 120C high, -100C low).
* The user asked multiple follow-up questions, including the population size of cheese miners in Lunapolis.
* The AI provided responses to each question, but it was later revealed that the information about cheese miners is not relevant to the initial query.

## ARTIFACTS
None

## NEXT STEPS
The specific tasks remaining to be completed include providing accurate and relevant information about the capital of the moon.


## Trim/delete messages

In [8]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage


@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]

    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [9]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
    temperature=0.0,
)

agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [10]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='981cce79-9043-47f9-9428-9113f4e7d1a0'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='53a567d3-398d-4214-8540-9eeee9894673', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='a7b209d2-aab5-4665-a7bf-c7dd59ce2171'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='f96357dd-91e0-4c2d-a06b-844c4f0a8443', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='ef283214-167c-4dd9-bba1-d94e38548964'),
              AIMessage(content='That\'s a good question! If your device is not turning on, che

> [!NOTE] 
> We can see that the tool messages have been removed from the state.

If the `ToolMessages` were removed before reaching the agent, it should not know the correct temperature.

In [11]:
print(response["messages"][-1].content)

That's a good question! If your device is not turning on, checking the temperature might help you identify the issue.

If it's extremely hot (above 90°C/194°F) or freezing cold (below -20°C/-4°F), it could be related to overheating or freezing. Try letting it cool down or warming it up slightly before trying to turn it on again.

However, if the temperature is within a normal range, here are some next steps:

1. **Charge it**: If your device has removable batteries, try charging them for at least 30 minutes.
2. **Check for physical damage**: Inspect the device for any signs of physical damage, such as cracks or dents.
3. **Restart in safe mode**: Some devices have a "safe mode" that can help diagnose issues. Check your device's manual or online documentation to see if this option is available.
4. **Try a hard reset**: If your device has a removable battery, try taking it out and putting it back in. This can sometimes resolve software-related issues.
5. **Contact the manufacturer or a r